In [2]:
# Use autoplot 207's code for this app
import datetime
import sys

import numpy as np
from pyproj import Transformer

import geopandas as gpd
import pandas as pd
from iemweb.autoplot.scripts200.p207 import USEME, add_zeros, compute_grid_bounds, do_analysis, load_data
from matplotlib.patches import Rectangle
from pyiem.nws.vtec import NWS_COLORS
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_sqlalchemy_conn, utc
from shapely.geometry import Point

In [20]:
STORM_NUMBER = 1
TITLE = "2 December 2024"
SUBTITLE = "7 AM 3 December 2024"
SETPOINT_LOCS = {}
sts = utc(2024, 12, 2, 12)
ets = utc(2024, 12, 27, 18)
# Get available data
ctx = {
    "coop": "yes",
    "t": "state",
    "sz": 30,
    "z": "yes",
    "f": "linear",
    "v": "snow",
    "wfo": "DMX",
}
df = load_data(ctx, sts, ets)
df = df[~df["nwsli"].isin(["DSXI4", "DMX"])]
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, "IA")
# add zeros and QC
df = add_zeros(df, ctx)

In [4]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
            """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
            conn,
            params=(sts - datetime.timedelta(days=1), ets),
        )
    print(df[df["state"] == "IA"])
    mp.plot_values(
        df.lon.values,
        df.lat.values,
        df.magnitude.values,
        fmt="%.2f",
        labelbuffer=1,
        color="purple",
    )


def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE,),
        subtitle=(
            "Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
            "IEM 2022-2023 Winter Storm #%s"
        )
        % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2["val"] >= lower) & (df2["val"] < upper)]
    print(df[df["state"] == "IA"])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(
            df_useme_plot["lon"],
            df_useme_plot["lat"],
            df_useme_plot["val"].values,
            "%s",
            labels=df_useme_plot["nwsli"].values,
            textsize=10,
            labeltextsize=10,
            labelbuffer=1,
        )
    return mp

In [5]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(
            SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1]
        )
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True


def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.0
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz / 2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz / 2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1


def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets),
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0,), ("Snow Squall Warning",), loc=1).set_zorder(
        1000
    )

In [5]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,NM,ABQ,40.0,-105.450000,36.600000,POINT (-487406.329 -917399.256),True,0,True,LSR,-24,-12
1,NM,ABQ,34.0,-105.800000,35.790000,POINT (-524331.326 -1004977.452),True,1,True,LSR,-26,-15
2,NM,ABQ,31.0,-105.450000,36.580000,POINT (-487539.000 -919614.306),False,2,False,LSR,-24,-12
3,MN,DLH,21.6,-92.230000,46.700000,POINT (591445.548 217636.208),True,3,True,LSR,11,25
4,NM,ABQ,21.0,-105.550000,36.160000,POINT (-499296.308 -965536.738),True,4,True,LSR,-25,-14
...,...,...,...,...,...,...,...,...,...,...,...,...
2288,KY,LMK,0.0,-86.233100,37.793600,POINT (1205313.784 -701144.584),True,CNRK2,True,COOP,31,-5
2289,KY,LMK,0.0,-85.165800,38.429200,POINT (1285853.858 -615279.085),True,NWCK2,True,COOP,34,-2
2290,KY,LMK,0.0,-84.833600,38.219600,POINT (1318224.535 -633096.517),True,RVEK2,True,COOP,35,-3
2291,KY,LMK,0.0,-86.216700,36.733300,POINT (1224686.912 -817539.729),True,SCTK2,True,COOP,32,-9


In [22]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = [
        'IA-CD-11',
        'IA-CD-6',
        'POCI4',
        'EMMI4',
    ]
    if cull:
        df.loc[df["nwsli"].isin(cull), USEME] = False
    hardcode = []
    for nwsli, val in hardcode:
        df.loc[df["nwsli"] == nwsli, "val"] = val

    ctx["csector"] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0, upper=2)
    #draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename="241203.png")
    mp.close()


main()

     state  wfo  val        lon        lat                             geo  \
199     IA  DMX  2.0 -94.210000  42.190000  POINT (476669.595 -295590.336)   
201     IA  DMX  2.0 -94.220000  42.510000  POINT (473399.445 -260138.315)   
218     IA  DMX  1.6 -92.380000  41.030000  POINT (638431.263 -411812.949)   
220     IA  DMX  1.5 -92.590000  41.600000  POINT (615385.968 -350247.875)   
221     IA  DMX  1.5 -93.860000  41.610000  POINT (510103.755 -357836.698)   
...    ...  ...  ...        ...        ...                             ...   
1195    IA  DMX  0.0 -92.731031  41.746466  POINT (602315.977 -335059.172)   
1196    IA  DMX  0.0 -92.903614  41.127947  POINT (593796.869 -404875.444)   
1199    IA  DMX  0.0 -92.205655  40.610224  POINT (657223.783 -456949.387)   
1202    IA  DMX  0.0 -94.965020  41.494340  POINT (419220.955 -376916.846)   
1203    IA  ARX  0.0 -91.875100  42.680100  POINT (662946.976 -224987.271)   

      used_for_analysis     nwsli  plotme source  xcell  ycell 